# Инициализация

Загружаем библиотеки необходимые для выполнения кода ноутбука.

In [9]:
import pandas as pd
import boto3
from io import BytesIO
import pandas as pd
import os


# === ЭТАП 1 ===

# Загрузка первичных данных

Загружаем первичные данные из файлов:
- tracks.parquet
- catalog_names.parquet
- interactions.parquet

In [10]:


# Загрузка данных
tracks = pd.read_parquet('tracks.parquet')
catalog_names = pd.read_parquet('catalog_names.parquet')
interactions = pd.read_parquet('interactions.parquet')

# Обзор данных

Проверяем данные, есть ли с ними явные проблемы.

In [11]:
print(tracks.info())
print(tracks.isnull().sum())
print(tracks['track_id'].nunique() == len(tracks))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 4 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   track_id  1000000 non-null  int64 
 1   albums    1000000 non-null  object
 2   artists   1000000 non-null  object
 3   genres    1000000 non-null  object
dtypes: int64(1), object(3)
memory usage: 30.5+ MB
None
track_id    0
albums      0
artists     0
genres      0
dtype: int64
True


In [12]:
print(catalog_names.info())
print(catalog_names.isnull().sum())
print(catalog_names.groupby('type')['id'].nunique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1812471 entries, 0 to 1812470
Data columns (total 3 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   id      int64 
 1   type    object
 2   name    object
dtypes: int64(1), object(2)
memory usage: 41.5+ MB
None
id      0
type    0
name    0
dtype: int64
type
album      658724
artist     153581
genre         166
track     1000000
Name: id, dtype: int64


In [13]:
print(interactions.info())
print(interactions.isnull().sum())
print(interactions.duplicated(subset=['user_id', 'track_id']).sum())

<class 'pandas.core.frame.DataFrame'>
Index: 222629898 entries, 0 to 291
Data columns (total 4 columns):
 #   Column      Dtype         
---  ------      -----         
 0   user_id     int32         
 1   track_id    int32         
 2   track_seq   int16         
 3   started_at  datetime64[ns]
dtypes: datetime64[ns](1), int16(1), int32(2)
memory usage: 5.4 GB
None
user_id       0
track_id      0
track_seq     0
started_at    0
dtype: int64
0


# Выводы

Приведём выводы по первому знакомству с данными:
- есть ли с данными явные проблемы,
- какие корректирующие действия (в целом) были предприняты.

# === ЭТАП 2 ===

# EDA

Распределение количества прослушанных треков.

In [14]:
# Группируем по пользователям и считаем количество прослушиваний
user_plays = interactions.groupby('user_id').size().reset_index(name='tracks_played')
distribution = user_plays['tracks_played'].describe(percentiles=[.25, .5, .75, .9, .95, .99])
print("Распределение количества прослушанных треков:")
print(distribution)

Распределение количества прослушанных треков:
count    1.373221e+06
mean     1.621224e+02
std      3.512846e+02
min      1.000000e+00
25%      2.300000e+01
50%      5.500000e+01
75%      1.540000e+02
90%      3.890000e+02
95%      6.500000e+02
99%      1.576000e+03
max      1.663700e+04
Name: tracks_played, dtype: float64


Наиболее популярные треки

In [15]:
# Считаем количество прослушиваний для каждого трека
track_popularity = interactions.groupby('track_id').size().reset_index(name='plays')
top_tracks = track_popularity.sort_values('plays', ascending=False).head(10)
print("\nТоп-10 популярных треков:")
print(top_tracks)


Топ-10 популярных треков:
        track_id   plays
9098       53404  111062
483876  33311009  106921
26665     178529  101924
512157  35505245   99490
829320  65851540   86670
368072  24692821   86246
475289  32947997   85886
696106  51241318   85244
90461     795836   85042
647237  45499814   84748


Наиболее популярные жанры

In [16]:

track_genres = tracks.set_index('track_id')['genres'].to_dict()



In [17]:

track_plays = interactions['track_id'].value_counts().reset_index()
track_plays.columns = ['track_id', 'plays']

In [18]:
from collections import defaultdict 

genre_plays = defaultdict(int)

In [19]:

for track_id, plays in track_plays.itertuples(index=False):
    genres = track_genres.get(track_id, [])
    for genre_id in genres:
        genre_plays[genre_id] += plays

In [20]:

genre_popularity = (
    pd.DataFrame(list(genre_plays.items()), columns=['genre_id', 'plays'])
    .sort_values('plays', ascending=False)
    .head(10)
)

In [21]:

genre_names = (
    catalog_names[catalog_names['type'] == 'genre']
    .set_index('id')['name']
    .to_dict()
)

genre_popularity['genre_name'] = genre_popularity['genre_id'].map(genre_names)

print("Топ-10 популярных жанров:")
print(genre_popularity[['genre_name', 'plays']])

Топ-10 популярных жанров:
     genre_name     plays
7           pop  55578312
6           rap  37799821
2       allrock  31092013
8        ruspop  26626241
5        rusrap  25303695
14  electronics  20120981
11        dance  16291557
19      rusrock  13166147
1          rock  12772644
4         metal  12437375


Треки, которые никто не прослушал

In [22]:
# Находим треки из каталога, отсутствующие в прослушиваниях
unplayed_tracks = tracks[~tracks['track_id'].isin(interactions['track_id'])][['track_id']]
print("\nТреки без прослушиваний:")
print(unplayed_tracks)


Треки без прослушиваний:
Empty DataFrame
Columns: [track_id]
Index: []


# Преобразование данных

Преобразуем данные в формат, более пригодный для дальнейшего использования в расчётах рекомендаций.

In [23]:
# Извлечение названий треков
track_names = catalog_names[catalog_names['type'] == 'track'][['id', 'name']]
track_names = track_names.rename(columns={'id': 'track_id', 'name': 'track_name'})

# Извлечение названий альбомов
album_names = catalog_names[catalog_names['type'] == 'album'][['id', 'name']]
album_names = album_names.rename(columns={'id': 'album_id', 'name': 'album_name'})

# Извлечение названий артистов
artist_names = catalog_names[catalog_names['type'] == 'artist'][['id', 'name']]
artist_names = artist_names.rename(columns={'id': 'artist_id', 'name': 'artist_name'})

# Извлечение названий жанров
genre_names = catalog_names[catalog_names['type'] == 'genre'][['id', 'name']]
genre_names = genre_names.rename(columns={'id': 'genre_id', 'name': 'genre_name'})

# Объединение данных
items = (
    tracks
    # Добавление названий треков
    .merge(track_names, on='track_id', how='left')
    
    # Развертывание списков альбомов
    .explode('albums')
    .rename(columns={'albums': 'album_id'})
    # Добавление названий альбомов
    .merge(album_names, on='album_id', how='left')
    
    # Развертывание списков артистов
    .explode('artists')
    .rename(columns={'artists': 'artist_id'})
    # Добавление названий артистов
    .merge(artist_names, on='artist_id', how='left')
    
    # Развертывание списков жанров
    .explode('genres')
    .rename(columns={'genres': 'genre_id'})
    # Добавление названий жанров
    .merge(genre_names, on='genre_id', how='left')
    
    # Группировка в списки
    .groupby(['track_id', 'track_name']).agg({
        'album_id': list,
        'album_name': list,
        'artist_id': list,
        'artist_name': list,
        'genre_id': list,
        'genre_name': list
    }).reset_index()
)

# Проверка результата
print(items.head(3))

   track_id        track_name                  album_id   
0        26  Complimentary Me  [3, 3, 2490753, 2490753]  \
1        38       Momma's Boy  [3, 3, 2490753, 2490753]   
2       135           Atticus        [12, 214, 2490809]   

                                          album_name         artist_id   
0  [Taller Children, Taller Children, Taller Chil...  [16, 16, 16, 16]  \
1  [Taller Children, Taller Children, Taller Chil...  [16, 16, 16, 16]   
2  [Wild Young Hearts, Wild Young Hearts, Wild Yo...      [84, 84, 84]   

                                         artist_name          genre_id   
0  [Elizabeth & the Catapult, Elizabeth & the Cat...  [11, 21, 11, 21]  \
1  [Elizabeth & the Catapult, Elizabeth & the Cat...  [11, 21, 11, 21]   
2                  [Noisettes, Noisettes, Noisettes]      [11, 11, 11]   

               genre_name  
0  [pop, folk, pop, folk]  
1  [pop, folk, pop, folk]  
2         [pop, pop, pop]  


# Сохранение данных

Сохраним данные в двух файлах в персональном S3-бакете по пути `recsys/data/`:
- `items.parquet` — все данные о музыкальных треках,
- `events.parquet` — все данные о взаимодействиях.

In [24]:
events = interactions.copy()

In [25]:
%load_ext autoreload
%autoreload 2
from dotenv import load_dotenv, find_dotenv
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
# подгружаем .env
load_dotenv()

True

In [27]:
s3_bucket = os.environ.get('S3_BUCKET_NAME')
s3_access_key = os.environ.get('AWS_ACCESS_KEY_ID')
s3_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')

In [28]:
print(s3_access_key)

YCAJE3Nlz8iDILW5VTYM1ihQB


In [31]:

# Проверка переменных окружения
AWS_ACCESS_KEY_ID = s3_access_key
AWS_SECRET_ACCESS_KEY = s3_secret_access_key
BUCKET_NAME = s3_bucket

# Проверка наличия всех переменных окружения
if not all([AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY, BUCKET_NAME]):
    missing = [var for var in ["AWS_ACCESS_KEY_ID", "AWS_SECRET_ACCESS_KEY", "S3_BUCKET_NAME"] 
               if not os.getenv(var)]
    raise ValueError(f"Не установлены переменные окружения: {', '.join(missing)}")

# Инициализация клиента S3
s3 = boto3.client('s3',
                  aws_access_key_id=AWS_ACCESS_KEY_ID,
                  aws_secret_access_key=AWS_SECRET_ACCESS_KEY)

def save_df_to_s3_parquet(df, bucket, key):
    """Сохраняет DataFrame в Parquet на S3"""
    # Проверка типа данных
    if not isinstance(df, pd.DataFrame):
        raise TypeError("df должен быть pandas DataFrame")
    
    if not isinstance(bucket, str):
        raise TypeError("bucket должен быть строкой")
    
    if not isinstance(key, str):
        raise TypeError("key должен быть строкой")
    
    buffer = BytesIO()
    df.to_parquet(buffer)
    buffer.seek(0)
    
    s3.put_object(Bucket=bucket, Key=key, Body=buffer.getvalue())
    print(f"Файл {key} успешно загружен в S3")

try:
    # Сохранение items.parquet
    save_df_to_s3_parquet(items, BUCKET_NAME, 'recsys/data/items.parquet')
    
    # Сохранение events.parquet
    save_df_to_s3_parquet(events, BUCKET_NAME, 'recsys/data/events.parquet')
    
    print("Все файлы успешно загружены в S3!")
except Exception as e:
    print(f"Ошибка при загрузке в S3: {e}")
    # Дополнительная диагностика
    print(f"Тип BUCKET_NAME: {type(BUCKET_NAME)}, значение: '{BUCKET_NAME}'")
    print(f"Тип items: {type(items)}")
    print(f"Тип events: {type(events)}")

Ошибка при загрузке в S3: An error occurred (InvalidAccessKeyId) when calling the PutObject operation: The AWS Access Key Id you provided does not exist in our records.
Тип BUCKET_NAME: <class 'str'>, значение: 's3-student-mle-20250130-b8bcb9fce1'
Тип items: <class 'pandas.core.frame.DataFrame'>
Тип events: <class 'pandas.core.frame.DataFrame'>


In [30]:

items.to_parquet("items.parquet")
events.to_parquet("events.parquet")

# Очистка памяти

Здесь, может понадобится очистка памяти для высвобождения ресурсов для выполнения кода ниже. 

Приведите соответствующие код, комментарии, например:
- код для удаление более ненужных переменных,
- комментарий, что следует перезапустить kernel, выполнить такие-то начальные секции и продолжить с этапа 3.

# === ЭТАП 3 ===

# Загрузка данных

Если необходимо, то загружаем items.parquet, events.parquet.

# Разбиение данных

Разбиваем данные на тренировочную, тестовую выборки.

# Топ популярных

Рассчитаем рекомендации как топ популярных.

# Персональные

Рассчитаем персональные рекомендации.

# Похожие

Рассчитаем похожие, они позже пригодятся для онлайн-рекомендаций.

# Построение признаков

Построим три признака, можно больше, для ранжирующей модели.

# Ранжирование рекомендаций

Построим ранжирующую модель, чтобы сделать рекомендации более точными. Отранжируем рекомендации.

# Оценка качества

Проверим оценку качества трёх типов рекомендаций: 

- топ популярных,
- персональных, полученных при помощи ALS,
- итоговых
  
по четырем метрикам: recall, precision, coverage, novelty.

# === Выводы, метрики ===

Основные выводы при работе над расчётом рекомендаций, рассчитанные метрики.